In [3]:
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from specbuffer import ReplayBuffer
from PPOee import PPO
import gym
from CustomEnv import CustomEnv
from Student_Agent import StudentAgent
from TeacherAgent import TeacherAgent
from ExpertTrajectory import ExpertTrajectory

import contextlib
import network_sim
from stable_baselines3 import PPO as sb3ppo
import pickle

cuda:0


2024-03-31 14:08:01.889424: I tensorflow/stream_executor/platform/default/dso_loader.cc:50] Successfully opened dynamic library libcudart.so.12


In [4]:
env=CustomEnv('PccNs-v0',1200)

History length: 10
Features: ['sent latency inflation', 'latency ratio', 'send ratio']
Getting min obs for ['sent latency inflation', 'latency ratio', 'send ratio']


In [10]:
et=ExpertTrajectory(1600)

In [11]:
et.load_expert_model('verygood.zip')
et.generate_trajectory(env,40)

  0%|          | 0/40 [00:00<?, ?it/s]

400it [00:00, 814.82it/s]             


In [5]:
class Student(StudentAgent):
    def __init__(self, state_dim, action_dim, env):
        super().__init__(state_dim, action_dim, env)
        
    def generate_trajectory(self, step: int):
        pb=tqdm(range(step))
        for _ in pb:
            s=self.env.reset()
            d=False
            while not d:
                a,action,l,v=self.model.select_action(s)
                s_,r,d,_=self.env.step(a)
                self.model.buffer.store(s[-3:],a,l,s_,r,v,d)
                s=s_
            pb.update()

In [6]:
sa=Student(30,1,env)

In [7]:
sa.generate_trajectory(10)

100%|██████████| 10/10 [00:07<00:00,  1.35it/s]


In [8]:
sa.train(10,8)

  0%|          | 0/10 [00:00<?, ?it/s]/home/tools/DR/MyProject/RILE/specbuffer.py:88: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  state=[torch.FloatTensor(self.get_state_padding(self.state[int(i/length)],i%length,10,True)[0]).view(-1) for i in index]
100%|██████████| 10/10 [00:01<00:00,  8.59it/s]


In [40]:
class Teacher(TeacherAgent):
    def __init__(self, state_dim, action_dim, env:CustomEnv, expert_trajectory):
        super().__init__(state_dim, action_dim, env, expert_trajectory)
        
    def ComputeReward(self):
        pb=tqdm(range(min(self.trajectory_buffer.index,self.trajectory_buffer.buffer_size)))
        for i in pb:
            length=len(self.trajectory_buffer.state[0])
            s=torch.FloatTensor(self.trajectory_buffer.get_state_padding(self.trajectory_buffer.state[int(i/length)],i%length,10,True)[1]).view(-1)
            sa_pair=torch.cat((s,self.trajectory_buffer.action[i]),-1)
            reward,_,l,v=self.model.select_action(sa_pair)
            self.replay_buffer.store(
                sa_pair,
                reward,
                l,
                self.trajectory_buffer.next_state[i],
                self.discriminator.model(sa_pair).detach().cpu().numpy(),
                v,
                self.trajectory_buffer.done[i]
            )
            self.trajectory_buffer.reward[i]=reward
            pb.update()
        self.discriminator.collect_expert()

In [41]:
ta=Teacher(30,1,env,et)

In [42]:
ta.ComputeReward()

100%|██████████| 1200/1200 [00:00<00:00, 1416.92it/s]


In [44]:
ta.discriminator.sbuffer.from_after_state([0,1]).cuda()

tensor([[9.1664e-17, 1.0000e+00, 1.0490e+00, 0.0000e+00, 1.0000e+00, 1.0909e+00,
         0.0000e+00, 1.0000e+00, 1.1212e+00, 0.0000e+00, 1.0000e+00, 1.0909e+00,
         0.0000e+00, 1.0000e+00, 1.0588e+00, 0.0000e+00, 1.0000e+00, 1.0588e+00,
         0.0000e+00, 1.0000e+00, 1.0278e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00,
         0.0000e+00, 1.0000e+00, 1.0909e+00, 0.0000e+00, 1.0000e+00, 1.0286e+00],
        [0.0000e+00, 1.0000e+00, 1.0909e+00, 0.0000e+00, 1.0000e+00, 1.1212e+00,
         0.0000e+00, 1.0000e+00, 1.0909e+00, 0.0000e+00, 1.0000e+00, 1.0588e+00,
         0.0000e+00, 1.0000e+00, 1.0588e+00, 0.0000e+00, 1.0000e+00, 1.0278e+00,
         0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00, 1.0909e+00,
         0.0000e+00, 1.0000e+00, 1.0286e+00, 0.0000e+00, 1.0000e+00, 1.0286e+00]],
       device='cuda:0')

In [47]:
ta.trainPPO(10,8)

ValueError: only one element tensors can be converted to Python scalars

In [38]:
import importlib
module=importlib.import_module('Discriminator','Discriminator.py')


In [39]:
d=module.Discriminator(31,1,8,sa.replay_buffer,et)

In [34]:
d.collect_expert()

In [35]:
d.collect_dist(8)

In [37]:
d.update(100,False)